<div style="display: flex; justify-content: center; align-items: center; height: 150px; background-color: #f3f4f6; border: 2px solid #4CAF50; border-radius: 10px; padding: 10px; text-align: center; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);">
    <div style="text-align: center;">
        <h1 style="color: #4CAF50; margin: 10px; font-size: 2.5em;">Assignment | Operation Research</h1>
    </div>
    <div style="text-align: left; margin-left: 20px;">
        <h3 style="color: #333; margin: 10px 0 0;">Name: Subhraneil Das</h3>
        <h3 style="color: #333; margin: 5px 0 0;">PRN: 23070243053</h3>
    </div>
</div>

## Importing Libraries

In [4]:
import numpy as np
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

## A. Linear Programming Problem 

Q. Carpenter manufactures two products. Product A generates a profit of Rs.23, B generates profit of Rs.10. It takes 2 hours to produce Product A and 0.8 to produce Product B. He can't spend more than 25 hours per week and total number of units for A & B cannot be greater than 20. Find the number of units of A & B to maximize his profit.

In [5]:
# Create a linear programming maximization problem
prob = LpProblem("Carpenter_Profit_Maximization", LpMaximize)

# Decision variables
x1 = LpVariable('x1', lowBound=0, cat='Continuous')  # Number of product A
x2 = LpVariable('x2', lowBound=0, cat='Continuous')  # Number of product B

# Objective function (maximize profit)
prob += 23 * x1 + 10 * x2, "Profit"

# Constraints
prob += 2 * x1 + 0.8 * x2 <= 25, "TimeConstraint"
prob += x1 + x2 <= 20, "ProductionConstraint"

# Solve the problem
prob.solve()

# Get the results
optimal_x1 = x1.varValue
optimal_x2 = x2.varValue
optimal_profit = prob.objective.value()

print(f"Optimal number of units of Product A: {optimal_x1}")
print(f"Optimal number of units of Product B: {optimal_x2}")
print(f"Maximum profit: {optimal_profit}")

Optimal number of units of Product A: 7.5
Optimal number of units of Product B: 12.5
Maximum profit: 297.5


## B. Transportation Problem

### 1. North-West Method

In [9]:
def north_west_corner_with_cost(cost_matrix, supply, demand):
    n, m = len(supply), len(demand)
    allocation = [[0] * m for _ in range(n)]
    
    i = j = 0
    total_cost = 0
    while i < n and j < m:
        alloc = min(supply[i], demand[j])
        allocation[i][j] = alloc
        total_cost += alloc * cost_matrix[i][j]  # Calculate the cost
        supply[i] -= alloc
        demand[j] -= alloc
        
        if supply[i] == 0:
            i += 1
        if demand[j] == 0:
            j += 1
    
    return allocation, total_cost

# Example cost matrix, supply, and demand
cost_matrix = [
    [3, 1, 7, 4],
    [2, 6, 5, 9],
    [8, 3, 3, 2]
]
supply = [300, 400, 500]  # supply
demand = [250, 350, 400, 200]  # demand

# Solving the transportation problem
allocation, total_cost = north_west_corner_with_cost(cost_matrix, supply, demand)

print("North-West Corner Method Allocation:")
for row in allocation:
    print(row)

print("\nTotal Transportation Cost:", total_cost)


North-West Corner Method Allocation:
[250, 50, 0, 0]
[0, 300, 100, 0]
[0, 0, 300, 200]

Total Transportation Cost: 4400


### 2. Least Cost Method

In [12]:
def least_cost_method(cost, supply, demand):
    n, m = len(supply), len(demand)
    allocation = [[0] * m for _ in range(n)]
    total_cost = 0  # Initialize the total cost
    
    while sum(supply) > 0 and sum(demand) > 0:
        min_cost = float('inf')
        min_i = min_j = 0
        
        # Find the minimum cost cell
        for i in range(n):
            for j in range(m):
                if supply[i] > 0 and demand[j] > 0 and cost[i][j] < min_cost:
                    min_cost = cost[i][j]
                    min_i, min_j = i, j
        
        # Allocate as much as possible
        alloc = min(supply[min_i], demand[min_j])
        allocation[min_i][min_j] = alloc
        total_cost += alloc * min_cost  # Calculate and add to the total cost
        supply[min_i] -= alloc
        demand[min_j] -= alloc
    
    return allocation, total_cost

# Example cost matrix, supply, and demand
cost_matrix = [
    [3, 1, 7, 4],
    [2, 6, 5, 9],
    [8, 3, 3, 2]
]
supply = [300, 400, 500]  # supply
demand = [250, 350, 400, 200]  # demand

allocation, total_cost = least_cost_method(cost_matrix, supply, demand)

print("Least Cost Method Allocation:")
for row in allocation:
    print(row)

print("\nTotal Transportation Cost:", total_cost)

Least Cost Method Allocation:
[0, 300, 0, 0]
[250, 0, 150, 0]
[0, 50, 250, 200]

Total Transportation Cost: 2850


### 3. Vogel's Approximation Method

Defining the supply demand and respective costs

In [2]:
grid = [[3, 1, 7, 4], [2, 6, 5, 9], [8, 3, 3, 2]]  # table
supply = [300, 400, 500]  # supply
demand = [250, 350, 400, 200]  # demand
INF = 10**3
n = len(grid)
m = len(grid[0])
ans = 0

Function to find the feasible solution

In [3]:
# hepler function for finding the row difference and the column difference
def findDiff(grid):
    rowDiff = []
    colDiff = []
    for i in range(len(grid)):
        arr = grid[i][:]
        arr.sort()
        rowDiff.append(arr[1]-arr[0])
    col = 0
    while col < len(grid[0]):
        arr = []
        for i in range(len(grid)):
            arr.append(grid[i][col])
        arr.sort()
        col += 1
        colDiff.append(arr[1]-arr[0])
    return rowDiff, colDiff


# loop runs until both the demand and the supply is exhausted
while max(supply) != 0 or max(demand) != 0:
    # finding the row and col difference
    row, col = findDiff(grid)
    # finding the maxiumum element in row difference array
    maxi1 = max(row)
    # finding the maxiumum element in col difference array
    maxi2 = max(col)

    # if the row diff max element is greater than or equal to col diff max element
    if(maxi1 >= maxi2):
        for ind, val in enumerate(row):
            if(val == maxi1):
                # finding the minimum element in grid index where the maximum was found in the row difference
                mini1 = min(grid[ind])
                for ind2, val2 in enumerate(grid[ind]):
                    if(val2 == mini1):
                        # calculating the min of supply and demand in that row and col
                        mini2 = min(supply[ind], demand[ind2])
                        ans += mini2 * mini1
                        # subtracting the min from the supply and demand
                        supply[ind] -= mini2
                        demand[ind2] -= mini2
                        # if demand is smaller then the entire col is assigned max value so that the col is eliminated for the next iteration
                        if(demand[ind2] == 0):
                            for r in range(n):
                                grid[r][ind2] = INF
                        # if supply is smaller then the entire row is assigned max value so that the row is eliminated for the next iteration
                        else:
                            grid[ind] = [INF for i in range(m)]
                        break
                break
    # if the row diff max element is greater than col diff max element
    else:
        for ind, val in enumerate(col):
            if(val == maxi2):
                # finding the minimum element in grid index where the maximum was found in the col difference
                mini1 = INF
                for j in range(n):
                    mini1 = min(mini1, grid[j][ind])

                for ind2 in range(n):
                    val2 = grid[ind2][ind]
                    if val2 == mini1:
                        # calculating the min of supply and demand in that row and col
                        mini2 = min(supply[ind2], demand[ind])
                        ans += mini2 * mini1
                        # subtracting the min from the supply and demand
                        supply[ind2] -= mini2
                        demand[ind] -= mini2
                        # if demand is smaller then the entire col is assigned max value so that the col is eliminated for the next iteration
                        if(demand[ind] == 0):
                            for r in range(n):
                                grid[r][ind] = INF
                        # if supply is smaller then the entire row is assigned max value so that the row is eliminated for the next iteration
                        else:
                            grid[ind2] = [INF for i in range(m)]
                        break
                break

print("The basic feasible solution using Vogel's Approximation Method is:", ans)

The basic feasible solution using Vogel's Approximation Method is: 2850


## C. Assignment Problem (Hungarian Method)

In [7]:
from scipy.optimize import linear_sum_assignment

# Example cost matrix
cost_matrix = np.array([
    [120, 100, 80, 150],
    [100, 85, 70, 115],
    [90, 100, 80, 140],
    [95, 120, 75, 180]
])

# Use SciPy's built-in Hungarian method
row_ind, col_ind = linear_sum_assignment(cost_matrix)

# Display the results
print("Optimal assignment pairs:", list(zip(row_ind, col_ind)))
print("Total minimum cost using Assignment Problem:", cost_matrix[row_ind, col_ind].sum())

Optimal assignment pairs: [(0, 1), (1, 3), (2, 0), (3, 2)]
Total minimum cost using Assignment Problem: 380
